In [1]:
import numpy as np
import glob
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pyemma
import msmbuilder
from msmbuilder.msm import MarkovStateModel
from msmbuilder.lumping import PCCAPlus

In [2]:
ls

dist.ipynb
pipeline-torque.sh
pyemma.log
run_pipeline.py
setd8_distances000000.pdb
setd8_distances000001.pdb
setd8_distances000002.pdb
setd8_distances000003.pdb
setd8_distances000004.pdb
setd8_distances000005.pdb
setd8_distances000006.pdb
setd8_distances000007.pdb
setd8_distances000008.pdb
setd8_distances000009.pdb
setd8_distances000010.pdb
setd8_distances000011.pdb
setd8_distances000012.pdb
setd8_distances000013.pdb
setd8_distances000014.pdb
setd8_distances000015.pdb
setd8_distances000016.pdb
setd8_distances000017.pdb
setd8_distances000018.pdb
setd8_distances000019.pdb
setd8_distances_dtrajs.npy
setd8_distances_macrostate_free_energies.npy
setd8_distances_macrostate_free_energies.png
setd8_distances_tica_kinetic_variance.png
setd8_distances_tica_projection.npy
setd8_distances_timescales_above_threshold.png
setd8_distances_transmat.npy


In [37]:
dtrajs = np.load('setd8_distances_dtrajs.npy', encoding='bytes')

In [3]:
dtrajs = list(dtrajs)

In [4]:
len(dtrajs)

3400

In [6]:
its = pyemma.msm.its(dtrajs)

02-02-17 15:45:25 pyemma.msm.estimators.implied_timescales.ImpliedTimescales[0] WARNING  Some timescales could not be computed. Timescales array is smaller than expected or contains NaNs


In [7]:
plt.figure()
pyemma.plots.plot_implied_timescales(its)
plt.savefig('its.png', dpi=300)
plt.close()

In [8]:
# let's try 200

In [9]:
msm_200 = MarkovStateModel(lag_time=200)
msm_200.fit(dtrajs)

MSM contains 57 strongly connected components above weight=0.01. Component 55 selected, with population 98.475880%


MarkovStateModel(ergodic_cutoff='on', lag_time=200, n_timescales=None,
         prior_counts=0, reversible_type='mle', sliding_window=True,
         verbose=True)

In [10]:
msm = msm_200
project = 'msm_200'

statdist = msm.populations_
relative_counts = msm.countsmat_.sum(0) / np.sum(msm.countsmat_)

plt.figure()
plt.scatter(statdist, relative_counts)
plt.xlabel('MSM stationary distribution')
plt.ylabel('Relative counts')
plt.savefig('sanity_check_%s.png' %project, dpi=300)
plt.close()

In [11]:
# try p1

In [13]:
msm_200_p1 = MarkovStateModel(lag_time=200, ergodic_cutoff=0.1)
msm_200_p1.fit(dtrajs)

MSM contains 182 strongly connected components above weight=0.10. Component 174 selected, with population 93.259335%


MarkovStateModel(ergodic_cutoff=0.1, lag_time=200, n_timescales=None,
         prior_counts=0, reversible_type='mle', sliding_window=True,
         verbose=True)

In [14]:
msm = msm_200_p1
project = 'msm_200_p1'

statdist = msm.populations_
relative_counts = msm.countsmat_.sum(0) / np.sum(msm.countsmat_)

plt.figure()
plt.scatter(statdist, relative_counts)
plt.xlabel('MSM stationary distribution')
plt.ylabel('Relative counts')
plt.savefig('sanity_check_%s.png' %project, dpi=300)
plt.close()

In [15]:
msm_200_p2 = MarkovStateModel(lag_time=200, ergodic_cutoff=0.2)
msm_200_p2.fit(dtrajs)

MSM contains 243 strongly connected components above weight=0.20. Component 223 selected, with population 89.638950%


MarkovStateModel(ergodic_cutoff=0.2, lag_time=200, n_timescales=None,
         prior_counts=0, reversible_type='mle', sliding_window=True,
         verbose=True)

In [16]:
msm = msm_200_p2
project = 'msm_200_p2'

statdist = msm.populations_
relative_counts = msm.countsmat_.sum(0) / np.sum(msm.countsmat_)

plt.figure()
plt.scatter(statdist, relative_counts)
plt.xlabel('MSM stationary distribution')
plt.ylabel('Relative counts')
plt.savefig('sanity_check_%s.png' %project, dpi=300)
plt.close()

In [5]:
msm_200_p3 = MarkovStateModel(lag_time=200, ergodic_cutoff=0.3)
msm_200_p3.fit(dtrajs)

MSM contains 293 strongly connected components above weight=0.30. Component 252 selected, with population 86.218707%


MarkovStateModel(ergodic_cutoff=0.3, lag_time=200, n_timescales=None,
         prior_counts=0, reversible_type='mle', sliding_window=True,
         verbose=True)

In [6]:
msm = msm_200_p3
project = 'msm_200_p3'

statdist = msm.populations_
relative_counts = msm.countsmat_.sum(0) / np.sum(msm.countsmat_)

plt.figure()
plt.scatter(statdist, relative_counts)
plt.xlabel('MSM stationary distribution')
plt.ylabel('Relative counts')
plt.savefig('sanity_check_%s.png' %project, dpi=300)
plt.close()

NameError: name 'msm_200_p3' is not defined

In [19]:
# guess that looks alright - let's see timescales and 20 macrostates

In [23]:
plt.figure()
plt.plot(msm_200_p3.timescales_, '.')
plt.xlim = -5
plt.yscale = 'log'
plt.savefig('timescales_msm_200_p3.png', dpi=300)
plt.close()

In [6]:
pcca_200_p3_20 = PCCAPlus.from_msm(msm_200_p3, n_macrostates=20)
dtrajs_200_p3_20 = pcca_200_p3_20.transform(dtrajs)

Optimization terminated successfully.
         Current function value: -19.399090
         Iterations: 22
         Function evaluations: 2557


In [7]:
# back here 4/21/17: need to extract traj, frame indices for all frames / macrostate, so I can study RMSD per macrostate

In [18]:
def metastable_sets(msmbuilder_pcca_object, msmbuilder_msm, dtrajs=None):
    import random
    import numpy as np
    
    # make dtrajs_dict
    dtrajs_dict = dict()
    traj_index = 0

    for traj in dtrajs:
        frame_index = 0
        for frame in traj:
            if frame in dtrajs_dict:
                dtrajs_dict[frame].append((traj_index, frame_index))
            else:
                dtrajs_dict[frame] = [(traj_index, frame_index)]
            frame_index += 1    
        traj_index += 1    

    metastable_sets = []
    samples = []
    
    # make mestable_sets
    for x in range(msmbuilder_pcca_object.n_macrostates):
        metastable_sets.append([])
    for x in range(len(msmbuilder_pcca_object.microstate_mapping_)):
        metastable_sets[msmbuilder_pcca_object.microstate_mapping_[x]].append(x)

    metastable_sets2 = []
    
    for metastable_set in metastable_sets:
        all_frames = []
        for microstate in metastable_set:
            microstate_label = msmbuilder_msm.state_labels_[microstate]
            all_frames += dtrajs_dict[microstate_label]
        metastable_sets2.append(all_frames)
        
    return metastable_sets2

In [19]:
meta_sets = metastable_sets(pcca_200_p3_20, msm_200_p3, dtrajs=dtrajs)

In [20]:
meta_sets

[[(811, 212),
  (811, 216),
  (2788, 186),
  (2788, 290),
  (2788, 292),
  (2788, 296),
  (2788, 297),
  (2788, 298),
  (2788, 299),
  (2788, 300),
  (2788, 301),
  (2788, 302),
  (2788, 303),
  (2788, 304),
  (2788, 305),
  (2788, 306),
  (2788, 307),
  (2788, 308),
  (2788, 309),
  (2788, 310),
  (2788, 311),
  (2788, 312),
  (2788, 313),
  (2788, 314),
  (2788, 315),
  (2788, 316),
  (2788, 317),
  (2788, 318),
  (2788, 319),
  (2788, 320),
  (2788, 321),
  (2788, 322),
  (2788, 323),
  (2788, 324),
  (2788, 325),
  (2788, 326),
  (2788, 327),
  (2788, 328),
  (2788, 329),
  (2788, 330),
  (2788, 331),
  (2788, 332),
  (2788, 333),
  (2788, 334),
  (2788, 335),
  (2788, 336),
  (2788, 337),
  (2788, 338),
  (2788, 339),
  (2788, 340),
  (2788, 341),
  (2788, 342),
  (2788, 343),
  (2788, 344),
  (2788, 345),
  (2788, 346),
  (2788, 347),
  (2788, 348),
  (2788, 349),
  (2788, 350),
  (2788, 351),
  (2788, 352),
  (2788, 353),
  (2788, 354),
  (2788, 355),
  (2788, 356),
  (2788, 357

In [21]:
len(meta_sets)

20

In [22]:
total = 0

for x in meta_sets:
    for y in x:
        total += 1

In [23]:
total

5809307

In [24]:
# 5.8 M frames - all good

In [25]:
pwd

'/Users/rafalpwiewiora/repos/MSM/11707_11709_SET8_apo/msm_pipeline/dist'

In [26]:
np.save('msm_200_p3_20_meta_sets', meta_sets)

In [36]:
len(dtrajs[0])

2050

In [7]:
# MSMBUILDER ONLY - SAMPLES EQUAL AMOUNTS OF FRAMES / MACROSTATE; 
def macrostate_sample_equal(msmbuilder_msm, msmbuilder_pcca_object, n_samples_per_macrostate=100, dtrajs=None):
    import random
    import numpy as np
    
    # make dtrajs_dict
    dtrajs_dict = dict()
    traj_index = 0

    for traj in dtrajs:
        frame_index = 0
        for frame in traj:
            if frame in dtrajs_dict:
                dtrajs_dict[frame].append((traj_index, frame_index))
            else:
                dtrajs_dict[frame] = [(traj_index, frame_index)]
            frame_index += 1    
        traj_index += 1    

    metastable_sets = []
    samples = []
    
    # make mestable_sets
    for x in range(msmbuilder_pcca_object.n_macrostates):
        metastable_sets.append([])
    for x in range(len(msmbuilder_pcca_object.microstate_mapping_)):
        metastable_sets[msmbuilder_pcca_object.microstate_mapping_[x]].append(x)

    # sample
    for metastable_set in metastable_sets:
        all_frames = []
        for microstate in metastable_set:
            microstate_label = msmbuilder_msm.state_labels_[microstate]
            all_frames += dtrajs_dict[microstate_label]
        macrostate_samples = random.sample(all_frames, n_samples_per_macrostate)
        samples.append(macrostate_samples)
            
    # samples have to have np.ndarrays in them, not just lists
    samples_ = []
    for x in samples:
        samples_.append(np.asarray(x))
    
    return samples_

In [26]:
def calculate_macrostate_means(rmsd_matrix, samples, plot_title, plot_path):
    macrostates = len(samples)

    macrostate_fragments_rmsd_matrix = []
    x = 0
    y = len(samples[0])

    for i in range(macrostates):
        fragment = rmsd_matrix[x:y, x:y]
        macrostate_fragments_rmsd_matrix.append(fragment)
        if not i == range(macrostates)[-1]:
            x += len(samples[i])
            y += len(samples[i+1])
    
    macrostate_fragments_rmsd_matrix_means = []
    for i in macrostate_fragments_rmsd_matrix:
        macrostate_fragments_rmsd_matrix_means.append(np.mean(i))
    
    plt.figure()
    plt.plot(macrostate_fragments_rmsd_matrix_means, '.')
    plt.title(plot_title)
    plt.xticks(range(0, macrostates, 5))
    plt.xlabel('Macrostate label')
    plt.ylabel('Mean intra-state RMSD (nm)')
    plt.savefig(plot_path, dpi=300)
    plt.close()    

In [27]:
samples_200_p3_20 = macrostate_sample_equal(msm_200_p3, pcca_200_p3_20, n_samples_per_macrostate=10, dtrajs=dtrajs)

In [7]:
msm_200_p3_20 = pyemma.msm.estimate_markov_model(dtrajs_200_p3_20, lag=200)

In [9]:
plt.figure()
pyemma.plots.plot_markov_model(msm_200_p3_20, arrow_labels=None)
plt.savefig('msm_200_p3_20.png', dpi=300)
plt.close()

In [8]:
plt.figure()
plt.spy(msm_200_p3_20.P)
plt.savefig('msm_200_p3_20_P.png', dpi=300)
plt.close()

In [8]:
# let's make a sparsity pattern of P array, to take to regspace_analysis to make a MSM-like figure 

In [10]:
msm_200_p3_20.P.shape

(20, 20)

In [11]:
msm_200_p3_20_spy = np.zeros((20,20))

In [12]:
for x in range(20):
    for y in range(20):
        if msm_200_p3_20.P[x,y] > 0:
            msm_200_p3_20_spy[x,y] = 1

In [13]:
msm_200_p3_20_spy

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         1.,  0.,  0.,  1.,  0.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,
         1.,  1.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  

In [10]:
msm_200_p3_20.stationary_distribution

array([ 0.00937809,  0.00511237,  0.00532692,  0.03194538,  0.01459391,
        0.01911732,  0.0277873 ,  0.00540315,  0.05325258,  0.01245424,
        0.02232931,  0.01905987,  0.01439557,  0.17017188,  0.08234085,
        0.04125468,  0.07237103,  0.21521267,  0.0586633 ,  0.11982958])

In [30]:
# taking samples to cluster

In [31]:
np.save('samples_200_p3_20', samples_200_p3_20)

In [32]:
# calc means but matrix not looking so good - I wanna repeat here and dih the rmsd matrix calc. WITHOUT N-FLANK

In [33]:
# calc means

In [34]:
rmsd_matrix = np.load('macro_sample/200_p3_20/rmsd_matrix_SET8_apo.npy')

In [35]:
calculate_macrostate_means(rmsd_matrix, samples_200_p3_20, 'SET8-apo dist 200_p3_20 macro_20', 'macro_sample/200_p3_20/means.png')

In [36]:
# now go back to cluster - calculate rmsd matrix without N-flank - done for dih and dist

In [37]:
# next - look at tica / tica per starting run here // try the no N-flank on RMSD-regspace and unitime as well

In [38]:
# next next - cluster - do all the RMSD analyses for Shi

In [41]:
# next next next - msm-pipeline no N-flank 50 // 100 // 95var for dist and dih+dist -- remember to remove chirality this time

In [40]:
# next next next next - unitime / reg-space clustering into 2.5k and 5k states of whatever tica's I have

In [43]:
rmsd_matrix = np.load('macro_sample/200_p3_20/rmsd_matrix_SET8_apo_noNflank.npy')
samples = np.load('macro_sample/200_p3_20/samples_200_p3_20.npy')

calculate_macrostate_means(rmsd_matrix, samples, 'SET8-apo dist 200_p3_20 macro_20', 'macro_sample/200_p3_20/means_noNflanK.png')

In [44]:
# ok looking at tica here

In [45]:
tica = np.load('setd8_distances_tica_projection.npy', encoding='bytes')

In [46]:
import msmexplorer as msme

In [47]:
data = np.concatenate(tica, axis=0)

In [48]:
data_5tics = data[:, :5]

In [49]:
plt.figure()
msme.plot_histogram(data_5tics, color='oxblood', quantiles=(0.5,),
                    labels=['$tIC1$', '$tIC2$', '$tIC3$', '$tIC4$', '$tIC5$'],
                    show_titles=True)
plt.savefig('5tics_hist.png', dpi=300)
plt.close()

In [50]:
data_2tics = data[:, :2]

In [51]:
plt.figure()
msme.plot_free_energy(data_2tics, obs=(0,1))
plt.savefig('2tics_plt.png', dpi=300)
plt.close()

In [52]:
x = data[:, 0]
y = data[:, 1]

plt.figure()
plt.scatter(x, y)
plt.savefig('2tics_scatter.png', dpi=300)
plt.close()

In [53]:
colors = np.load('../../msm_pipeline/dih/colors.npy')

In [54]:
colors

array(['red', 'red', 'red', ..., 'brown', 'brown', 'brown'], 
      dtype='<U6')

In [55]:
x = data[:, 0]
y = data[:, 1]

plt.figure()
plt.scatter(x, y, color=colors)
plt.savefig('2tics_scatter_color_by_run.png', dpi=300)
plt.close()

In [56]:
# now doing no-Nflank means for regspace and unitime as well
# then - RMSD diagnostics

In [58]:
# let's start 50 macrostates pcca here and then do RMSD diagnostics on the cluster - with and without N-flank

In [59]:
pcca_200_p3_50 = PCCAPlus.from_msm(msm_200_p3, n_macrostates=50)
dtrajs_200_p3_50 = pcca_200_p3_50.transform(dtrajs)

In [60]:
samples_200_p3_50 = macrostate_sample_equal(msm_200_p3, pcca_200_p3_50, n_samples_per_macrostate=10, dtrajs=dtrajs)

In [61]:
msm_200_p3_50 = pyemma.msm.estimate_markov_model(dtrajs_200_p3_50, lag=200)

In [66]:
plt.figure()
pyemma.plots.plot_markov_model(msm_200_p3_50)
plt.savefig('msm_200_p3_50.png', dpi=300)
plt.close()

/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/pyemma/plots/networks.py:70: UserWarning: The layout optimization method will take a long time for large networks! It is recommended to coarse grain your model first!
  warnings.warn("The layout optimization method will take a long"


In [64]:
np.save('samples_200_p3_50', samples_200_p3_50)

In [65]:
# taking samples to cluster

In [67]:
# calc means

In [68]:
rmsd_matrix = np.load('macro_sample/200_p3_50/rmsd_matrix_SET8_apo_noNflank.npy')
samples = np.load('macro_sample/200_p3_50/samples_200_p3_50.npy')

calculate_macrostate_means(rmsd_matrix, samples, 'SET8-apo', 'macro_sample/200_p3_50/means_noNflank.png')

In [9]:
msm_200_p3_20

NameError: name 'msm_200_p3_20' is not defined

In [8]:
# 50 macrostates

In [9]:
pcca_200_p3_50 = PCCAPlus.from_msm(msm_200_p3, n_macrostates=50)
dtrajs_200_p3_50 = pcca_200_p3_50.transform(dtrajs)

In [10]:
samples_200_p3_20_100per = macrostate_sample_equal(msm_200_p3, pcca_200_p3_20, n_samples_per_macrostate=100, dtrajs=dtrajs)

In [11]:
samples_200_p3_50_100per = macrostate_sample_equal(msm_200_p3, pcca_200_p3_50, n_samples_per_macrostate=100, dtrajs=dtrajs)

In [12]:
np.save('samples_200_p3_20_100per', samples_200_p3_20_100per)

In [13]:
np.save('samples_200_p3_50_100per', samples_200_p3_50_100per)

In [1]:
# back here to deal with unitime clustered 2500 & 5000 states - Feb 23 2017

In [9]:
pwd

'/Users/rafalpwiewiora/repos/MSM/11707_11709_SET8_apo/msm_pipeline/dist'